In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import pandas as pd
import sys
sys.path
sys.path.append('../models')
sys.path.append('../simulation')
import numpy as np
import pickle
import random
import os
import math
import sim_functions_cleaner  as sf
import operator
import study

In [2]:
def get_all_states():
    all_states = []
    counts = {}
    for person,data in experiment.population.items():
        h = data.history
        for time,cd in h.items():
            
            if cd['avail'] and cd['decision_time']:
                #cd['tod'],cd['dow'],cd['weather'],
                states = [1,cd['tod'],cd['dow'],cd['pretreatment'],cd['location']]
                key = '-'.join(str(i) for i in states)
                if key not in counts: 
                    counts[key]=0
                counts[key]=counts[key]+1
                all_states.append(states)
    return all_states

In [3]:
coeffs = np.array([ 0.05,  0.25, -0.3 ,  0.25,  0.25, -0.3 ])
coeffs = np.array([0.05,  0.25,  0.25,  0.25, -0.3 ])
#coeffs = np.array([ 0.05,  0.25, -0.3 ])

#coeffs = np.array([ 0.05, -0.3 ,  0.25,  0.25 ])

#coeffs = np.array([0.05,  0.3 ,  0.3 , -0.35 ])
#coeffs = np.array([ 0.05 , 0.25, -0.3])
#coeffs = np.array([0.05, ,  0.25, -0.3 ])

In [4]:
with open('../simulation/visited_states_200.pkl','rb') as f:
    experiment = pickle.load(f)

In [5]:
ast = get_all_states()


In [6]:
def get_all_states_by_group(gid):
    all_states = []
    counts = {}
    for person,data in experiment.population.items():
        h = data.history
        
        if data.gid==gid:
            for time,cd in h.items():
            
                if cd['avail'] and cd['decision_time']:
                    #cd['tod'],cd['dow'],cd['weather'],
                    states = [1,cd['tod'],cd['dow'],cd['pretreatment'],cd['location']]
                    key = '-'.join(str(i) for i in states)
                    if key not in counts: 
                        counts[key]=0
                    counts[key]=counts[key]+1
                    all_states.append(states)
    return all_states

In [7]:
group_two = get_all_states_by_group(2)
group_one = get_all_states_by_group(1)

In [8]:
def get_percent_actions(states,coeffs,offset):
    return [(np.dot(coeffs.T,s)+offset)>0 for s in states]

In [9]:
def get_reward(states,coeffs,offset):
    return [(np.dot(coeffs.T,i)+offset) for i in states]

In [10]:
def test_smooth(sigma):
    all_rewards = []
    #os = [np.random.normal(loc=-0.1,scale=.37) for i in range(8)]
    for i in range(50):
        offset = np.random.normal(loc=0.0,scale=sigma)
        #offset = os[i%8]
        #print(offset)
        rew = get_reward(ast,coeffs,offset)
        #return rew
        all_rewards.extend(rew)
    return all_rewards

In [11]:
#[np.random.normal(loc=-0.1,scale=.37) for i in range(8)]

In [12]:
#for s in np.arange(.1,.45,.05):
#    print(s)
#    rews = test_smooth(s)
#    print(sum([i for i in rews if i>=0])/len(rews))

In [397]:
arews = []
for a in range(10):
    rews = test_smooth(.37)
    arews.extend(rews)
print(sum([i for i in arews if i>=0])/len(arews))

0.31639962108926756


In [389]:
np.array(rews).mean()

0.28545013949279346

In [325]:
treat = get_reward(ast,coeffs,0)
print(np.array(treat).mean())

0.21751837694795645


In [169]:
coeffs = np.array([0.05,  0.25,  0.25,  -0.3, 0.25 ])

In [189]:
def get_percent_actions_three(states,coeffs,offset,offtwo):
    to_return = []
    for s in states:
        new_coeff = [i for i in coeffs]
        new_coeff[-1]=new_coeff[-1]+np.random.normal(-0.3,offset)
        #print(new_coeff)
        #print(s)
        to_return.append(np.dot(np.array(new_coeff).T,s)+np.random.normal(0.0,offtwo))
    return to_return

In [190]:
per_one = get_percent_actions_three(group_one,coeffs,0.25,.37)
per_two = get_percent_actions_three(group_two,coeffs,0.25,.37)

In [191]:
#print(sum(per_one)/len(per_one))
print(len([i for i in per_one+per_two if i>0])/len(per_one+per_two))

0.58582769773596


In [107]:
len([i for i in per_one+per_two if i>0])/len(per_one+per_two)

0.8844751543663628

In [110]:
np.array([(np.dot(coeffs.T,st)+0) for st in group_two+group_one]).std()

0.24442343298521066

In [111]:
sum(per_one)/len(per_one)

0.8219615530542558

In [112]:
sum(per_two)/len(per_two)

0.2120246207957793

In [ ]:
per_one = get_percent_actions(group_one,coeffs,0.0)
per_two = get_percent_actions(group_two,coeffs,-0.5)

In [159]:
coeffs_one = np.array([0.05,  0.25,  0.25,  -.3, .25+.25])
coeffs_two = np.array([0.05,  0.25,  0.25,  -.3, .25-.25])

In [162]:
rew_one = get_reward(group_one,coeffs_one,0.00)
rew_two = get_reward(group_two,coeffs_two,-.35)
per_one = get_percent_actions(group_one,coeffs_one,0.0)
per_two = get_percent_actions(group_two,coeffs_two,-.35)

In [163]:
print(len([i for i in per_one if i>0])/len(per_one))
print(len([i for i in per_two if i>0])/len(per_two))

0.7781712406728215
0.1048032534622994


In [164]:
alls = []
for s in group_one:
    alls.append(s[-1]*.25+.2)
for s in group_two:
    alls.append(s[-1]*.25-.2)

In [165]:
np.array(alls).std()

0.23341762211620554

In [664]:
np.array([np.random.normal(0,.323) for i in range(68020)])

0.32156081500853695

0.3341668498571114

In [216]:
#coeffs_one = np.array([.05,  0.0,  0.0,  0.0, -0.2+.25 ])
#coeffs_two = np.array([.05,  0.0,  0.0,  0.0, -0.2-.1 ])
#rew_one = get_reward(group_one,coeffs_one,-0.0)
#rew_two = get_reward(group_two,coeffs_two,-0.0)

In [166]:
np.array(rew_one+rew_two).std()

0.3709644326170883

In [219]:
for i in [5,25,50,75,95]:
    print(i)
    print(np.percentile([rew_one],i))

5
0.05
25
0.3
50
0.35
75
0.55
95
0.8


In [220]:
for i in [5,25,50,75,95]:
    print(i)
    print(np.percentile(rew_two,i))

5
-0.8500000000000001
25
-0.6000000000000001
50
-0.3
75
-0.04999999999999993
95
0.20000000000000007


In [221]:
np.array([r for r in rew_one+rew_two]).std()

0.446484362148451

In [370]:
for s in np.arange(.1,.45,.05):
    print(s)
    rews = test_smooth(s)
    print(sum([i for i in rews if i>=0])/len(rews))

0.15
0.22770362498378996
0.2
0.23374088956881384
0.25
0.26403586248768895
0.30000000000000004
0.3428470248066565
0.3500000000000001
0.27206724999040666
0.40000000000000013
0.3509350289657297
0.45000000000000007
0.382853039808708


In [198]:
-0.24999999999999994+.3

0.050000000000000044

In [199]:
0.050000000000000044/2

0.025000000000000022

In [539]:
np.percentile(rew_one+rew_two,50)

0.04999999999999999

In [203]:
.95--.5

1.45

In [204]:
1.45/4

0.3625

In [540]:
for i in [5,25,50,75,95]:
    print(i)
    print(np.percentile(rew_two+rew_one,i))

5
-0.55
25
-0.25
50
0.04999999999999999
75
0.05
95
0.35


In [206]:
.8--.5

1.3

In [207]:
1.3/4

0.325